In [3]:
import requests
from bs4 import BeautifulSoup
import time
from typing import List, Dict
import pandas as pd
import concurrent.futures
import re
import csv
import os

# Cấu hình Headers và các đường dẫn
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}
OUTPUT_CSV_FILE = 'batdongsan_listings.csv'

def sane_sleep():
    time.sleep(1)

def parse_number_from_text(text: str, unit: str):
    """Trích xuất giá trị số từ chuỗi văn bản."""
    if not text:
        return None
    s = text.lower().replace('.', '').replace(',', '.').strip()
    m = re.search(r'([0-9\.]+)\s*' + unit, s)
    if m:
        try:
            return float(m.group(1))
        except (ValueError, IndexError):
            return None
    return None

def scrape_list_page(url: str) -> List[Dict]:
    """Cạo dữ liệu cơ bản từ một trang danh sách."""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi truy cập trang {url}: {e}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    product_items = soup.find_all('div', class_='re__card-info')
    
    if not product_items:
        return []
    
    results = []
    for item in product_items:
        title_tag = item.find('a', class_='re__card-title')
        title = title_tag.get_text(strip=True) if title_tag else 'N/A'
        
        price_tag = item.find('span', class_='re__card-config-price')
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'
        
        area_tag = item.find('span', class_='re__card-config-area')
        area = area_tag.get_text(strip=True) if area_tag else 'N/A'

        url_tag = item.find('a', class_='re__card-title', href=True)
        detail_url = f"https://batdongsan.com.vn{url_tag['href']}" if url_tag and url_tag['href'].startswith('/') else url_tag['href'] if url_tag else 'N/A'

        results.append({
            'title': title,
            'price_raw': price,
            'area_raw': area,
            'detail_url': detail_url,
        })
    return results

def scrape_detail_page(listing: Dict) -> Dict:
    """Cạo dữ liệu chi tiết từ trang của một bất động sản."""
    detail_url = listing.get('detail_url')
    if not detail_url or detail_url == 'N/A':
        return listing

    try:
        response = requests.get(detail_url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        sane_sleep()
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi truy cập trang chi tiết {detail_url}: {e}")
        return listing

    soup = BeautifulSoup(response.content, 'html.parser')
    
    detailed_data = {
        'House Direction': None, 'Balcony Direction': None,
        'Bedrooms': None, 'Toilets': None, 'Legits': None,
        'Furnitures': None, 'Floors': None, 'Facade': None,
        'Entrance': None, 'City': None, 'District': None,
        'Ward': None, 'Street': None,
    }

    info_rows = soup.find_all('div', class_='re__pr-specs-content')
    for row in info_rows:
        key_tag = row.find('span', class_='re__pr-specs-label')
        value_tag = row.find('span', class_='re__pr-specs-value')
        
        if key_tag and value_tag:
            key = key_tag.get_text(strip=True).replace(':', '')
            value = value_tag.get_text(strip=True)

            if 'Hướng nhà' in key: detailed_data['House Direction'] = value
            elif 'Hướng ban công' in key: detailed_data['Balcony Direction'] = value
            elif 'Số phòng ngủ' in key: detailed_data['Bedrooms'] = value
            elif 'Số toilet' in key: detailed_data['Toilets'] = value
            elif 'Pháp lý' in key: detailed_data['Legits'] = value
            elif 'Nội thất' in key: detailed_data['Furnitures'] = value
            elif 'Số tầng' in key: detailed_data['Floors'] = value
            elif 'Mặt tiền' in key: detailed_data['Facade'] = value
            elif 'Đường' in key: detailed_data['Street'] = value

    address_tag = soup.find('div', class_='re__pr-short-info-content')
    if address_tag:
        address_parts = [part.strip() for part in address_tag.get_text(strip=True).split(',') if part.strip()]
        if len(address_parts) >= 1: detailed_data['Street'] = address_parts[0]
        if len(address_parts) >= 2: detailed_data['Ward'] = address_parts[-3]
        if len(address_parts) >= 3: detailed_data['District'] = address_parts[-2]
        if len(address_parts) >= 4: detailed_data['City'] = address_parts[-1]

    listing.update(detailed_data)
    return listing

def scrape_all_pages_and_details(base_url: str):
    """Cạo toàn bộ dữ liệu từ tất cả các trang và các trang chi tiết."""
    all_data = []
    page_number = 1
    
    while True:
        url = base_url if page_number == 1 else f"{base_url}/p{page_number}"
        print(f"Đang cạo dữ liệu từ trang {page_number}: {url}")
        
        page_data = scrape_list_page(url)
        if not page_data:
            print("Đã cạo xong tất cả các trang.")
            break
        
        all_data.extend(page_data)
        page_number += 1
        sane_sleep()
    
    print("\nBắt đầu cạo dữ liệu chi tiết của từng sản phẩm...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(scrape_detail_page, item): item for item in all_data}
        detailed_data = [future.result() for future in concurrent.futures.as_completed(futures)]

    processed_data = []
    for row in detailed_data:
        # Cập nhật giá và diện tích sau khi cạo chi tiết
        row['Price'] = parse_number_from_text(row.get('price_raw'), 'tỷ|triệu')
        row['Area'] = parse_number_from_text(row.get('area_raw'), 'm2')
        processed_data.append(row)

    return processed_data

def save_to_csv(data: List[Dict]):
    """Lưu danh sách dict vào file CSV."""
    if not data:
        print("Không có dữ liệu để lưu.")
        return

    columns = [
        'House Direction', 'Balcony Direction', 'Bedrooms', 'Toilets', 
        'Legits', 'Furnitures', 'Floors', 'Facade', 'Entrance', 
        'City', 'District', 'Ward', 'Street', 'Area', 'Price'
    ]

    df = pd.DataFrame(data)

    for col in columns:
        if col not in df.columns:
            df[col] = None
    df = df.reindex(columns=columns)
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_NONNUMERIC)
    print(f"\nĐã lưu thành công {len(data)} bản ghi vào file '{OUTPUT_CSV_FILE}'.")

if __name__ == '__main__':
    base_url = "https://batdongsan.com.vn/ban-nha-dat-ha-noi"
    
    all_scraped_data = scrape_all_pages_and_details(base_url)
    
    if all_scraped_data:
        save_to_csv(all_scraped_data)
    else:
        print("Không thể cạo dữ liệu.")

Đang cạo dữ liệu từ trang 1: https://batdongsan.com.vn/ban-nha-dat-ha-noi
Đang cạo dữ liệu từ trang 2: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p2
Đang cạo dữ liệu từ trang 3: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p3
Đang cạo dữ liệu từ trang 4: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p4
Đang cạo dữ liệu từ trang 5: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p5
Đang cạo dữ liệu từ trang 6: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p6
Đang cạo dữ liệu từ trang 7: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p7
Đang cạo dữ liệu từ trang 8: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p8
Đang cạo dữ liệu từ trang 9: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p9
Lỗi khi truy cập trang https://batdongsan.com.vn/ban-nha-dat-ha-noi/p9: 403 Client Error: Forbidden for url: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p9
Đã cạo xong tất cả các trang.

Bắt đầu cạo dữ liệu chi tiết của từng sản phẩm...


TypeError: float() argument must be a string or a real number, not 'NoneType'